Train

In [ ]:
!python ../../dump_dataset.py -o 5000 -c 110 -j 14

Test

In [14]:
!python ../../dump_dataset.py -o 1000 -c 70 -j 8

Set random seed to 8
100%|███████████████████████████████████████| 1000/1000 [00:31<00:00, 31.92it/s]


In [2]:
import tensorflow as tf

# Ruta al archivo TFRecord
file_path = 'sr_30/train_9_sr_30.tfrecord'

# Crear un dataset a partir del archivo TFRecord
raw_dataset = tf.data.TFRecordDataset(file_path)

# Función para parsear los ejemplos
# Función para parsear los ejemplos
def _parse_function(proto):
    # Especificar las características que esperas en el archivo TFRecord
    feature_description = {
        'inputs': tf.io.VarLenFeature(tf.float32),
        'sat': tf.io.VarLenFeature(tf.float32),
        'policy': tf.io.VarLenFeature(tf.float32),
    }
    parsed_features = tf.io.parse_single_example(proto, feature_description)
    # Convertir las características sparse a dense
    inputs = tf.sparse.to_dense(parsed_features['inputs'])
    sat = tf.sparse.to_dense(parsed_features['sat'])
    policy = tf.sparse.to_dense(parsed_features['policy'])
    return {'inputs': inputs, 'sat': sat, 'policy': policy}

# Aplicar la función de parseo al dataset
parsed_dataset = raw_dataset.map(_parse_function)

# Mostrar los primeros ejemplos del dataset
for raw_record in parsed_dataset.take(5):
    print(raw_record)

2024-07-17 19:24:24.615993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-17 19:24:24.675523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2024-07-17 19:24:24.675544: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-07-17 19:24:24.676141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with o

InvalidArgumentError: Key: inputs.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext]

In [ ]:
!tar -czvf neuroheurtf2_exp2.tar.gz ../../../neuroheurtf2

In [ ]:
!python ../../neurosat_tpu_tf2.py \
    --use_tpu=False \
    --train_file=sr_70/train_12_sr_70.tfrecord \
    --test_file=sr_70/test_8_sr_70.tfrecord \
    --train_steps=5000 \
    --test_steps=250 \
    --model_dir=model_sr_70 \
    --export_dir=export_sr_70 \
    --variable_number=70 \
    --clause_number=140 \
    --train_files_gzipped=False \
    --batch_size=64 \
    --export_model \
    --attention=True